# Business Problem

If the city of Minneaoplis could only afford 2 scooter maintenance facilities what 3 neighborhoods would be the best for building these facilities? The city of Minneapolis started a new scooter pilot program in 2019 to introduce a new form of public transportation. 

__Some of the main problems that we've seen from this pilot program are:__ 
- How to balance a limited suppply of scooters within the city so that we meet consumer demand
- How to ensure all scooters are charged throughout the day
- How to ensure there aren't an overuse of scooters congesting walkways and streets within the city

To solve these problems we would like to build scooter maintenance facilities around the city that are city owned and service all scooter types / scooter companies. 

__Projected benefits of Scooter facilities:__
 - House extra scooters for re-distribution
 - Balance the amount of scooters in a zone
 - Repair scooters
 - Charge scooters so there are less on the street
 - Create jobs
 
__Our criteria for deciding where to place these facilities:__
 - Scooter Congestion Ratio (SCR) of each neighborhood. (# of scooters / pop. density)
 - Number of transportation centers within each neighborhood (FrourSqure)
 - Average distance away from key transit locations (GeoCoder) 

# Data

Pilot program date range: May 17th, 2019 to January 14th, 2020

###### (243 days, or 7 months and 29 days.)

###### Scooter data: https://opendata.minneapolismn.gov/datasets/scooter-availability-2019/data?page=63408

# Methodology 

Import and install necessary libaries

In [1]:
pip install lxml

Note: you may need to restart the kernel to use updated packages.


In [2]:
import requests
import pandas as pd
import numpy as np
import sklearn.neighbors

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes #uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim #convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Folium installed')
print('Libraries imported.')

Solving environment: done

# All requested packages already installed.

Solving environment: done

# All requested packages already installed.

Folium installed
Libraries imported.


In [4]:
# Load in minneapolis scooter data
scooter_data = pd.read_csv('https://opendata.arcgis.com/datasets/47af2f88d31f45ccaaf945dbcc8f9665_0.csv')
scooter_data.head()

,PollTime,CompanyName,NumberAvailable,ClosestCenterlineID,ClosestCenterlineType,Neighborhood,ESRI_OID
0,2019/05/16 19:00:00+00,Lime,1,10122.00,street,Webber - Camden,1
1,2019/05/16 19:00:00+00,Lime,1,10464.00,street,Lind - Bohanon,2
2,2019/05/16 19:00:00+00,Lime,1,10801.00,street,Audubon Park,3
3,2019/05/16 19:00:00+00,Lime,1,11395.00,street,CARAG,4
4,2019/05/16 19:00:00+00,Lime,1,13067.00,street,Minnehaha,5


In [5]:
n = len(scooter_data.index)

print("Number of rows in table:", n)

Number of rows in table: 486378


In [6]:
u = len(pd.unique(scooter_data['CompanyName']))

print("Number of unique scooter companies in Minneapolis :", u)

Number of unique scooter companies in Minneapolis : 3


In [7]:
scooter_data['ClosestCenterlineType'].value_counts()

street    443889
trail      42489
Name: ClosestCenterlineType, dtype: int64

In [10]:
b = 443889 / 42489

print("How many times more likely a scooter was parked on the street:", b)

How many times more likely a scooter was parked on the street: 10.44715102732472


Using a few quick pandas analysis tools note that our scooter facility will need to serve __3 different scooter companies__. In addition, we find the majority of scooters were approximately __10.8 times more available on the street__ than scooters on a trail. This indicates we should build our facilities so that they're at least 10x more easily accessible by street. In this next section let's find which neighborhoods had the highest amount of available scooters. We begin this process by creating a new dataframe using the __drop__ function to eliminate all unecissary columns.

In [11]:
scooter_data1 = scooter_data.drop(['PollTime','PollTime', 'CompanyName', 'ClosestCenterlineID', 'ClosestCenterlineType','ESRI_OID'], axis=1)
scooter_data1

,NumberAvailable,Neighborhood
0,1,Webber - Camden
1,1,Lind - Bohanon
2,1,Audubon Park
3,1,CARAG
4,1,Minnehaha
...,...,...
486373,1,Steven's Square - Loring Heights
486374,1,Prospect Park - East River Road
486375,1,University of Minnesota
486376,1,Ericsson


In [12]:
# use group function to group data by neighborhood
scooter_data2 = scooter_data1.groupby(['Neighborhood']).sum()
scooter_data2

,NumberAvailable
Neighborhood,
Armatage,561
Audubon Park,2671
Bancroft,968
Beltrami,1387
Bottineau,10674
...,...
West Calhoun,4160
Whittier,20491
Willard - Hay,5986


In [13]:
# use sort function to show the number of available scooters in descending
scooter_data3 = scooter_data2.sort_values(by='NumberAvailable', ascending=False)
scooter_data3

,NumberAvailable
Neighborhood,
Downtown West,105251
Marcy Holmes,67993
University of Minnesota,52720
Cedar Riverside,41160
North Loop,39465
...,...
Morris Park,282
Kenny,207
Page,180


In [14]:
# list of 10 neighborhoods with greatest scooter availability
scooter_data3.nlargest(10, columns='NumberAvailable', keep='first')

,NumberAvailable
Neighborhood,
Downtown West,105251
Marcy Holmes,67993
University of Minnesota,52720
Cedar Riverside,41160
North Loop,39465
Prospect Park - East River Road,26602
Loring Park,24892
Como,24487
Holland,24380


In the section above we have ordered the neighborhoods in descending order from greatest to least number of avialable scooters. With this list we easily see the top 10 neighborhoods of highest scooter concentration. Next let's find the population density of each neighborhood by  __scraping__ wikipedia pages. This data will be useful in determining which of the top neighborhoods require a maintentance facility the most based on the number of people living within a neighborhood, and it's scooter availability. We can also get the zip code  of each neighborhood. Using __geocode__ along with each neighborhood's zip code we can then generate latitude and longitude values for each neighborhood.

In [15]:
url='https://en.wikipedia.org/wiki/Downtown_West,_Minneapolis'
df=pd.read_html(url, header=0)[0]
df

,Downtown West,Downtown West.1
0,Neighborhood,Neighborhood
1,Location of Downtown West within the U.S. city...,Location of Downtown West within the U.S. city...
2,Country,United States
3,State,Minnesota
4,County,Hennepin
5,City,Minneapolis
6,Community,Central
7,Area[1],Area[1]
8,• Total,0.759 sq mi (1.97 km2)
9,Population (2010)[2],Population (2010)[2]


In [16]:
url='https://en.wikipedia.org/wiki/Marcy-Holmes,_Minneapolis'
df=pd.read_html(url, header=0)[0]
df

,Marcy-Holmes,Marcy-Holmes.1
0,Neighborhood,Neighborhood
1,Location of Marcy-Holmes within the U.S. city ...,Location of Marcy-Holmes within the U.S. city ...
2,Country,United States
3,State,Minnesota
4,County,Hennepin
5,City,Minneapolis
6,Community,University
7,Area[1],Area[1]
8,• Total,0.825 sq mi (2.14 km2)
9,Population (2010)[2],Population (2010)[2]


In [17]:
url='https://en.wikipedia.org/wiki/University_(neighborhood),_Minneapolis'
df=pd.read_html(url, header=0)[0]
df

,University,University.1
0,Neighborhood,Neighborhood
1,The University of Minnesota's Minneapolis camp...,The University of Minnesota's Minneapolis camp...
2,Location of University within the U.S. city of...,Location of University within the U.S. city of...
3,Country,United States
4,State,Minnesota
5,County,Hennepin
6,City,Minneapolis
7,Community,University
8,Area[1],Area[1]
9,• Total,0.668 sq mi (1.73 km2)


In [18]:
url='https://en.wikipedia.org/wiki/Cedar-Riverside,_Minneapolis'
df=pd.read_html(url, header=0)[0]
df

,Cedar-RiversideWest Bank,Cedar-RiversideWest Bank.1
0,Neighborhood,Neighborhood
1,Nickname(s): Little Mogadishu[1],Nickname(s): Little Mogadishu[1]
2,Location of Cedar-Riverside within the U.S. ci...,Location of Cedar-Riverside within the U.S. ci...
3,Coordinates: 44°58′00″N 93°14′36″W﻿ / ﻿44.9666...,Coordinates: 44°58′00″N 93°14′36″W﻿ / ﻿44.9666...
4,Country,United States
5,State,Minnesota
6,County,Hennepin
7,City,Minneapolis
8,Community,University
9,Area[2],Area[2]


In [19]:
url='https://en.wikipedia.org/wiki/North_Loop,_Minneapolis'
df=pd.read_html(url, header=0)[0]
df

,"North LoopMarket Square, Market District","North LoopMarket Square, Market District.1"
0,Neighborhood,Neighborhood
1,Nickname(s): The Warehouse District,Nickname(s): The Warehouse District
2,Location of North Loop within the U.S. city of...,Location of North Loop within the U.S. city of...
3,Country,United States
4,State,Minnesota
5,County,Hennepin
6,City,Minneapolis
7,Community,Central
8,Area[1],Area[1]
9,• Total,0.837 sq mi (2.17 km2)


In [20]:
url='https://en.wikipedia.org/wiki/Prospect_Park,_Minneapolis'
df=pd.read_html(url, header=0)[0]
df

,Prospect ParkEast Minneapolis,Prospect ParkEast Minneapolis.1
0,Neighborhood,Neighborhood
1,"Prospect Park viewed from Stadium Village, wit...","Prospect Park viewed from Stadium Village, wit..."
2,"Nickname(s): The Angle, Tower Hill","Nickname(s): The Angle, Tower Hill"
3,Location of Prospect Park neighborhood within ...,Location of Prospect Park neighborhood within ...
4,Coordinates: 44°58′08″N 93°12′45″W﻿ / ﻿44.9688...,Coordinates: 44°58′08″N 93°12′45″W﻿ / ﻿44.9688...
5,Country,United States
6,State,Minnesota
7,County,Hennepin
8,City,Minneapolis
9,Community,University


In [21]:
# create dataframe with postal codes for each abbreviated neighborhood to see overlapping of postal codes 

data = {
    '55401': ['dtw', 'nl','x'],
    '55402': ['dtw','x','x'],
    '55403': ['dtw', 'nl','x'],
    '55404': ['dtw','x','x'],
    '55405': ['nl','x','x'],
    '55411': ['nl','x','x'],
    '55414': ['mh', 'u', 'pp'],
    '55415': ['dtw','x','x'],
    '55454': ['u', 'cr','x'],
    '55455': ['u','x','x'],
    '55487': ['dtw','x','x']}
    
    
postal_table = pd.DataFrame (data, columns = ['55401','55402','55403','55404','55405','55411','55414','55415','55454','55455','55487']
                   
                  )
postal_table


,55401,55402,55403,55404,55405,55411,55414,55415,55454,55455,55487
0,dtw,dtw,dtw,dtw,nl,nl,mh,dtw,u,u,dtw
1,nl,x,nl,x,x,x,u,x,cr,x,x
2,x,x,x,x,x,x,pp,x,x,x,x


From the table we can more easily visualize which postal codes overlap with multiple neighborhoods. In order to give each neighborhood a unique postal code so that each neighborhood can be assigned a unique coordinate, we manually assign the postal codes to each neighborhood so that doubling is minimized. If a neighborhood is uniquely assigned to a postal code, meaning, there are no other neighborhoods under a specific postal code, than this postal code is assigned to that neighborhood. For example, if a postal code was shared by multiple neighborhoods but there is only one unique neighborhood this postal code is assigned to than the unique neighborhood gets the postal code. The only neighborhoods that will share the same postal code are Marcy Holmes and Prospect Park, since these neighborhoods have the same unique postal code of 55414.

In [22]:
# For loop to print out latitude and longitude coordinates for each postal code
postal = ['55404', '55414', '55455', '55454', '55405', '55414'] 
for x in postal:
    address = x
    geolocator = Nominatim(user_agent="ny_explorer")
    location = geolocator.geocode(address)
    latitude = location.latitude
    longitude = location.longitude
    print(latitude, longitude)

44.96109575 -93.255894125
44.98711935 -93.24115775
44.97473309462231 -93.23407422863202
44.9676909 -93.23853524743203
44.957127150000005 -93.30278685186671
44.98711935 -93.24115775


Now we calculate the scooter congestion ratio (SCR). We are assuming a high population density correlates with more congested living as more people are packed into a smaller space. We can devide the number of scooters available by the population density for each neighborhood to get a "scooter congestion ratio." Scooter congestion is linked to cluttering of streets/sidewalks, higher scooter demand, and/or scooters in need of powering.

__Scooter congestion ratio (SCR) = (# of scooters) / (population density)__

In [23]:
scooters_list = [136050,91214,66801,52683,49052,36927]
pop_density_list = [2900,4700,3100,5700,2000,2300]

res = [i / j for i, j in zip(scooters_list, pop_density_list)]
print(res)

[46.91379310344828, 19.407234042553192, 21.548709677419353, 9.242631578947368, 24.526, 16.055217391304346]


In [25]:
# create pandas dataframe to easily visualize which neighborhoods are most congested.

data = {'neighborhood': ['Downtown West','Marcy Holmes','University of Minnesota','Cedar Riverside','North Loop','Prospect Park'],
        'scooters':  ['105251', '67993','52720','41160','39465','26602'],
        'pop density (/km^2)': ['2,900', '4,700','3,100','5,700','2,000','2,300'],
        'SCR': ['46.914','19.407','21.549','9.243','24.526','16.055'],
        'Postal Code': ['55404', '55414', '55455', '55454', '55405', '55414'],
        'latitude': ['44.96109575', '44.98711935', '44.97473309462231', '44.9676909', '44.957127150000005', '44.9710'],
        'longitude': ['-93.255894125', '-93.24115775', '-93.23407422863205', '-93.23853524743203', '-93.30278685186671', '-93.2153']
       }

table1 = pd.DataFrame (data, columns = ['neighborhood','scooters','pop density (/km^2)', 'SCR', 'Postal Code','latitude', 'longitude'], 
                   index=['0','1','2','3','4','5']
                  )
table1

,neighborhood,scooters,pop density (/km^2),SCR,Postal Code,latitude,longitude
0,Downtown West,105251,"2,900",46.914,55404,44.96109575,-93.255894125
1,Marcy Holmes,67993,"4,700",19.407,55414,44.98711935,-93.24115775
2,University of Minnesota,52720,"3,100",21.549,55455,44.97473309462231,-93.23407422863205
3,Cedar Riverside,41160,"5,700",9.243,55454,44.9676909,-93.23853524743203
4,North Loop,39465,"2,000",24.526,55405,44.957127150000005,-93.30278685186671
5,Prospect Park,26602,"2,300",16.055,55414,44.9710,-93.2153


The highest SCR for each neighborhood in descending order is:
1. Downtown West (46.914)
2. North Loop (24.526)
3. University of_Minnesota (21.549)
4. Marcy Holmes (19.407)
5. Prospect Park (16.055)
6. Cedar Riverside (9.243)

## List of key locations

Let's find the coordinates to some of Minneapolis' known high traffic areas so we can consider them when choosing which 3 neighborhoods to place scooter maintenance facilities.

In [26]:
address = '300 Washington Ave SE, Minneapolis, MN 55455'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of central Univeristy of Minnesota {}, {}.'.format(latitude, longitude))

The geograpical coordinate of central Univeristy of Minnesota 44.97377352390003, -93.23580432404897.


In [27]:
address = '3001 Hennepin Ave, Minneapolis, MN 55408'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Calhoun Square {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Calhoun Square 44.9771885, -93.2759196.


In [28]:
address = '4825 Minnehaha Ave, Minneapolis, MN 55417'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Minnehaha Falls {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Minnehaha Falls 44.91502861590828, -93.20671585069111.


In [29]:
address = '300 South 6th St, Minneapolis, MN 55487'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Central Downtown Minneapolis {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Central Downtown Minneapolis 44.97586835, -93.26664821257543.


## Finding average distance 

Now let's find the average total distance that each of the most congested neighborhoods (shown above) is from each of our key transit locations. These are locations where high commuter traffick is known. After finding these distances we will organize each neighborhood in a table from lowest average distance to heighest. This table will then be a deciding factor in which neighborhood to place a maintenance facility.  

In [30]:
# Create two dataframes with city names and lat-long in degrees
table2 = pd.DataFrame({
    'neighborhood' :     ['Downtown West','Marcy Holmes','University','Cedar Riverside','North Loop','Prospect Park'],
    'latitude':  [ 44.96109575, 44.98711935, 44.97473309462231, 44.9676909, 44.957127150000005, 44.9710],
    'longitude': [ -93.255894125, -93.24115775, -93.23407422863205, -93.23853524743203, -93.30278685186671, -93.2153]
})
table3 = pd.DataFrame({
    'location':      ['University of Minnesota','Calhoun Square','Minnehaha Falls','Central Downtown Minneapolis'],
    'latitude_B' : [ 44.97377352390003, 44.9771885, 44.91502861590828,44.97586835],
    'longitude_B': [ -93.23580432404897, -93.2759196, -93.20671585069111,-93.26664821257543]
})

In [31]:
# add columns with radians for latitude and longitude
table2[['lat_radians','long_radians']] = (
    np.radians(table2.loc[:,['latitude','longitude']])
)
table3[['lat_radians_B','long_radians_B']] = (
    np.radians(table3.loc[:,['latitude_B','longitude_B']])
)

In [32]:
table2

,neighborhood,latitude,longitude,lat_radians,long_radians
0,Downtown West,44.961096,-93.255894,0.784719,-1.627622
1,Marcy Holmes,44.987119,-93.241158,0.785173,-1.627365
2,University,44.974733,-93.234074,0.784957,-1.627242
3,Cedar Riverside,44.967691,-93.238535,0.784834,-1.627319
4,North Loop,44.957127,-93.302787,0.784650,-1.628441
5,Prospect Park,44.971000,-93.215300,0.784892,-1.626914


In [33]:
table3

,location,latitude_B,longitude_B,lat_radians_B,long_radians_B
0,University of Minnesota,44.973774,-93.235804,0.784940,-1.627272
1,Calhoun Square,44.977188,-93.275920,0.785000,-1.627972
2,Minnehaha Falls,44.915029,-93.206716,0.783915,-1.626764
3,Central Downtown Minneapolis,44.975868,-93.266648,0.784977,-1.627810


Find distance between neighborhood and key location measured in miles. We will use the library __sklearn__ to use the 'haversine' distance, which assumes the world is a true sphere with a radius equal to 1. This makes computation easy and doesn't include the 3rd dimension of height when calculating a distance between two coordinates. To get the distance in miles we muliply the output of the sklearn computation by 3959 miles (the average radius of the earth). To get the distance in Km we could use a number of 6371. 

In [34]:
dist = sklearn.neighbors.DistanceMetric.get_metric('haversine')
dist_matrix = (dist.pairwise
    (table2[['lat_radians','long_radians']],
     table3[['lat_radians_B','long_radians_B']])*3959
)
# Note that 3959 is the radius of the earth in miles
df_dist_matrix = (
    pd.DataFrame(dist_matrix,index=table2['neighborhood'], 
                 columns=table3['location'])
)

In [35]:
df_dist_matrix

location,University of Minnesota,Calhoun Square,Minnehaha Falls,Central Downtown Minneapolis
neighborhood,,,,
Downtown West,1.316042,1.481499,3.989778,1.148182
Marcy Holmes,0.958567,1.832313,5.258326,1.468509
University,0.107463,2.052421,4.336987,1.594161
Cedar Riverside,0.440987,1.941718,3.957663,1.485886
North Loop,3.470829,1.909640,5.526684,2.190510
Prospect Park,1.020468,2.993869,3.890208,2.532446


In [36]:
df_dist_matrix['average'] = df_dist_matrix.mean(numeric_only=True, axis=1)
df_dist_matrix

location,University of Minnesota,Calhoun Square,Minnehaha Falls,Central Downtown Minneapolis,average
neighborhood,,,,,
Downtown West,1.316042,1.481499,3.989778,1.148182,1.983875
Marcy Holmes,0.958567,1.832313,5.258326,1.468509,2.379429
University,0.107463,2.052421,4.336987,1.594161,2.022758
Cedar Riverside,0.440987,1.941718,3.957663,1.485886,1.956564
North Loop,3.470829,1.909640,5.526684,2.190510,3.274416
Prospect Park,1.020468,2.993869,3.890208,2.532446,2.609248


In [37]:
table4 = df_dist_matrix.sort_values(by=['average'], ascending=True)
table4

location,University of Minnesota,Calhoun Square,Minnehaha Falls,Central Downtown Minneapolis,average
neighborhood,,,,,
Cedar Riverside,0.440987,1.941718,3.957663,1.485886,1.956564
Downtown West,1.316042,1.481499,3.989778,1.148182,1.983875
University,0.107463,2.052421,4.336987,1.594161,2.022758
Marcy Holmes,0.958567,1.832313,5.258326,1.468509,2.379429
Prospect Park,1.020468,2.993869,3.890208,2.532446,2.609248
North Loop,3.470829,1.909640,5.526684,2.190510,3.274416


From the table above in the 'average' column we see the nearest neighborhood to our key locations is Cedar Riverside. The next nearest neighborhood to all key locations is Downtown West. We can use this information later when deciding which neighborhoods to install the maintenance faciltiies.

## Using FourSquare API

In this section we will explore the number of transit centers within each neighborhood using the __FourSquare API__. Building a scooter maintenance facility next to or within a transit center is a priority because commuters will be able to conveniently connect to busess, trains, etc.

Since Cedar Riverside is the nearest neighborhood to all key locations (as shown above), we chose this neighborhood as our center point for using the FourSquare API. Next we defined FourSqure to look for assets containing "transit center," and that are within x amount of radial meters from our center point.

In [38]:
# Define my FourSquare credentials and version.
# Note that with a free account we are limited to a maximum of 50 calls (limit = 50). 

CLIENT_ID = 'EAJJX1MODHA1XCGUIZ5VRU4BU04XAN0DW0BCPE1RM2EBMQVW' # your Foursquare ID
CLIENT_SECRET = 'LDJRGGX2DZWOWBCUUU2M4CKKWLXXAGDCD2RL20SUFUWI2SOI' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 50
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: EAJJX1MODHA1XCGUIZ5VRU4BU04XAN0DW0BCPE1RM2EBMQVW
CLIENT_SECRET:LDJRGGX2DZWOWBCUUU2M4CKKWLXXAGDCD2RL20SUFUWI2SOI


In [39]:
# define search. Radius in meters. Latitude & longitude are the location of Cedar Riverside neighborhood.
search_query = 'Transit Center'
radius = 8000
print(search_query + ' .... OK!')

latitude = 44.967691
longitude = -93.238535

Transit Center .... OK!


In [40]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=EAJJX1MODHA1XCGUIZ5VRU4BU04XAN0DW0BCPE1RM2EBMQVW&client_secret=LDJRGGX2DZWOWBCUUU2M4CKKWLXXAGDCD2RL20SUFUWI2SOI&ll=44.967691,-93.238535&v=20180604&query=Transit Center&radius=8000&limit=50'

In [41]:
# call FourSquare API to display results based on search query above
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5fa56b96f0d3381728e72816'},
 'response': {'venues': [{'id': '4bf29a70a32e20a1f261d557',
    'name': 'Rosedale Transit Center',
    'location': {'address': '10 Rosedale Ctr',
     'lat': 45.012738055886814,
     'lng': -93.16888077269982,
     'labeledLatLngs': [{'label': 'display',
       'lat': 45.012738055886814,
       'lng': -93.16888077269982}],
     'distance': 7430,
     'postalCode': '55113',
     'cc': 'US',
     'city': 'Roseville',
     'state': 'MN',
     'country': 'United States',
     'formattedAddress': ['10 Rosedale Ctr',
      'Roseville, MN 55113',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fe931735',
      'name': 'Bus Station',
      'pluralName': 'Bus Stations',
      'shortName': 'Bus Station',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/travel/busstation_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1604676502',
    'hasPerk': False},
   {'id': '4b5c9c

In [42]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet,venuePage.id,location.neighborhood
0,4bf29a70a32e20a1f261d557,Rosedale Transit Center,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1604676502,False,10 Rosedale Ctr,45.012738,-93.168881,"[{'label': 'display', 'lat': 45.01273805588681...",7430,55113,US,Roseville,MN,United States,"[10 Rosedale Ctr, Roseville, MN 55113, United ...",NaN,NaN,NaN
1,4b5c9cdaf964a520e83929e3,Fairview Transitional Center,"[{'id': '4bf58dd8d48988d196941735', 'name': 'H...",v-1604676502,False,2512 S 7th St,44.967045,-93.235336,"[{'label': 'display', 'lat': 44.96704483032226...",261,55454,US,Minneapolis,MN,United States,"[2512 S 7th St, Minneapolis, MN 55454, United ...",NaN,NaN,NaN
2,50170b1fe4b0c3d592d66512,5th Street North Transit Center,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1604676502,False,NaN,44.980843,-93.275128,"[{'label': 'display', 'lat': 44.98084329574028...",3232,55403,US,Minneapolis,MN,United States,"[Minneapolis, MN 55403, United States]",NaN,NaN,NaN
3,4d9ce4957865a0934ea3db7a,Ramp B/ 5th St Transit Center,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1604676502,False,516 2nd Ave N,44.981507,-93.275898,"[{'label': 'display', 'lat': 44.98150713161008...",3320,55403,US,Minneapolis,MN,United States,"[516 2nd Ave N (I-394), Minneapolis, MN 55403,...",I-394,NaN,NaN
4,4dfa968b149557de37aaa336,Columbia Heights Transit Center,"[{'id': '4bf58dd8d48988d1fe931735', 'name': 'B...",v-1604676502,False,4079 Central Ave NE,45.042417,-93.246944,"[{'label': 'display', 'lat': 45.04241675562383...",8344,55421,US,Columbia Heights,MN,United States,"[4079 Central Ave NE, Columbia Heights, MN 554...",NaN,NaN,NaN


In [43]:
# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Rosedale Transit Center,Bus Station,10 Rosedale Ctr,45.012738,-93.168881,"[{'label': 'display', 'lat': 45.01273805588681...",7430,55113,US,Roseville,MN,United States,"[10 Rosedale Ctr, Roseville, MN 55113, United ...",NaN,NaN,4bf29a70a32e20a1f261d557
1,Fairview Transitional Center,Hospital,2512 S 7th St,44.967045,-93.235336,"[{'label': 'display', 'lat': 44.96704483032226...",261,55454,US,Minneapolis,MN,United States,"[2512 S 7th St, Minneapolis, MN 55454, United ...",NaN,NaN,4b5c9cdaf964a520e83929e3
2,5th Street North Transit Center,Bus Station,NaN,44.980843,-93.275128,"[{'label': 'display', 'lat': 44.98084329574028...",3232,55403,US,Minneapolis,MN,United States,"[Minneapolis, MN 55403, United States]",NaN,NaN,50170b1fe4b0c3d592d66512
3,Ramp B/ 5th St Transit Center,Bus Station,516 2nd Ave N,44.981507,-93.275898,"[{'label': 'display', 'lat': 44.98150713161008...",3320,55403,US,Minneapolis,MN,United States,"[516 2nd Ave N (I-394), Minneapolis, MN 55403,...",I-394,NaN,4d9ce4957865a0934ea3db7a
4,Columbia Heights Transit Center,Bus Station,4079 Central Ave NE,45.042417,-93.246944,"[{'label': 'display', 'lat': 45.04241675562383...",8344,55421,US,Columbia Heights,MN,United States,"[4079 Central Ave NE, Columbia Heights, MN 554...",NaN,NaN,4dfa968b149557de37aaa336
5,Veteran's Transition Center,College Classroom,"101 Pleasant St SE, University of Minnesota - ...",44.975821,-93.236447,"[{'label': 'display', 'lat': 44.9758205822622,...",919,55455,US,Minneapolis,MN,United States,"[101 Pleasant St SE, University of Minnesota -...",NaN,NaN,4f53fb16e4b0e14ed8ba6f83
6,MN State Fair Main Gate Transit Center,Bus Station,NaN,44.980698,-93.165090,"[{'label': 'display', 'lat': 44.980698, 'lng':...",5962,55108,US,Saint Paul,MN,United States,"[Saint Paul, MN 55108, United States]",NaN,NaN,57ce4e3e498e532c6b0e8a7a
7,"University of Minnesota Medical Center, Fairvi...",Hospital,2450 Riverside Ave,44.967756,-93.238135,"[{'label': 'display', 'lat': 44.96775643983586...",32,55454,US,Minneapolis,MN,United States,"[2450 Riverside Ave, Minneapolis, MN 55454, Un...",NaN,NaN,4aae58a9f964a520e36120e3
8,U of M-Fairview Riverside- professional Center,Medical Center,606 24th Ave S,44.967232,-93.238108,"[{'label': 'display', 'lat': 44.96723199690378...",61,55454,US,Minneapolis,MN,United States,"[606 24th Ave S (Riverside), Minneapolis, MN 5...",Riverside,NaN,4c96aa63bc279c7433bf6853
9,Maternal-Fetal Medicine Center,Doctor's Office,606 24th Ave S Ste 400,44.967129,-93.238068,"[{'label': 'display', 'lat': 44.96712875366211...",72,55454,US,Minneapolis,MN,United States,"[606 24th Ave S Ste 400, Minneapolis, MN 55454...",NaN,NaN,4bdf2269e75c0f47edffc903


In [44]:
dataframe_filtered.name

0                               Rosedale Transit Center
1                          Fairview Transitional Center
2                       5th Street North Transit Center
3                         Ramp B/ 5th St Transit Center
4                       Columbia Heights Transit Center
5                           Veteran's Transition Center
6                MN State Fair Main Gate Transit Center
7     University of Minnesota Medical Center, Fairvi...
8        U of M-Fairview Riverside- professional Center
9                        Maternal-Fetal Medicine Center
10                              Gateway Transit Station
11                         Chicago-Lake Transit Station
12                       Willey Hall Metro Transit Stop
13                               Uptown Transit Station
14                           U of M Medical Center Ramp
15    University of Minnesota Health Clinics and Sur...
16                     Children's Rehabilitation Center
17    Kennedy Center: James Haglund Family Fitne

In [45]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around neighborhood

# add a red circle marker to represent the neighborhood
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='Conrad Hotel',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# search query as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

In [46]:
n = len(dataframe_filtered.index)

print("Number of rows in table:", n)

Number of rows in table: 50


In [47]:
#To select rows whose column value equals a scalar, some_value, use ==:
df2 = dataframe_filtered.loc[dataframe_filtered['categories'] == 'Bus Station']
df2

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,neighborhood,id
0,Rosedale Transit Center,Bus Station,10 Rosedale Ctr,45.012738,-93.168881,"[{'label': 'display', 'lat': 45.01273805588681...",7430,55113,US,Roseville,MN,United States,"[10 Rosedale Ctr, Roseville, MN 55113, United ...",NaN,NaN,4bf29a70a32e20a1f261d557
2,5th Street North Transit Center,Bus Station,NaN,44.980843,-93.275128,"[{'label': 'display', 'lat': 44.98084329574028...",3232,55403,US,Minneapolis,MN,United States,"[Minneapolis, MN 55403, United States]",NaN,NaN,50170b1fe4b0c3d592d66512
3,Ramp B/ 5th St Transit Center,Bus Station,516 2nd Ave N,44.981507,-93.275898,"[{'label': 'display', 'lat': 44.98150713161008...",3320,55403,US,Minneapolis,MN,United States,"[516 2nd Ave N (I-394), Minneapolis, MN 55403,...",I-394,NaN,4d9ce4957865a0934ea3db7a
4,Columbia Heights Transit Center,Bus Station,4079 Central Ave NE,45.042417,-93.246944,"[{'label': 'display', 'lat': 45.04241675562383...",8344,55421,US,Columbia Heights,MN,United States,"[4079 Central Ave NE, Columbia Heights, MN 554...",NaN,NaN,4dfa968b149557de37aaa336
6,MN State Fair Main Gate Transit Center,Bus Station,NaN,44.980698,-93.165090,"[{'label': 'display', 'lat': 44.980698, 'lng':...",5962,55108,US,Saint Paul,MN,United States,"[Saint Paul, MN 55108, United States]",NaN,NaN,57ce4e3e498e532c6b0e8a7a
10,Gateway Transit Station,Bus Station,3rd St,44.979020,-93.261520,"[{'label': 'display', 'lat': 44.97901980661695...",2206,55415,US,Minneapolis,MN,United States,"[3rd St (5th Ave), Minneapolis, MN 55415, Unit...",5th Ave,NaN,4c3e26da80bc20a1c99caa58
11,Chicago-Lake Transit Station,Bus Station,2950 Chicago Ave S,44.949676,-93.262592,"[{'label': 'display', 'lat': 44.94967599765591...",2759,55407,US,Minneapolis,MN,United States,"[2950 Chicago Ave S (btw Lake & 29th St), Minn...",btw Lake & 29th St,NaN,4b10955ff964a520457323e3
13,Uptown Transit Station,Bus Station,2860 Hennepin Ave S,44.950196,-93.298415,"[{'label': 'display', 'lat': 44.95019627835658...",5103,55408,US,Minneapolis,MN,United States,"[2860 Hennepin Ave S (at W 29th St), Minneapol...",at W 29th St,NaN,4acd318af964a52055cb20e3
28,Metro Transit Wellness Center Stop,Bus Station,NaN,44.978163,-93.229896,"[{'label': 'display', 'lat': 44.97816284161655...",1349,55414,US,Minneapolis,MN,United States,"[Minneapolis, MN 55414, United States]",NaN,NaN,500f2242e4b00fee85cf27c5
29,Metro Transit - Emerald St Westbound,Bus Station,University,44.968395,-93.208237,"[{'label': 'display', 'lat': 44.96839487552816...",2387,NaN,US,Saint Paul,MN,United States,"[University (Emerald), Saint Paul, MN, United ...",Emerald,NaN,4cf3d07588de3704ea37812b


In [48]:
df2['postalCode'].value_counts()

55403    2
55408    1
55407    1
55108    1
55421    1
55414    1
55113    1
55415    1
Name: postalCode, dtype: int64

Based on the counts above the only applicable postal codes shown are 55403 (Downtwon West, North Loop), and 55414 (University). If we are to rank which neighborhoods have the most "transit stations" it would look like this: 
- Downtown West (2)
- North Loop (2)
- University (1)

We see Downton West and North Loop are tied for the top rank with two counts, and Univeristy is ranked last with only one count. 

# Neighborhood Ranking 

In [8]:
data = {
        'availability_rank': [6.0,5.0,4.0,3.0,2.0,1.0],
        'SCR_rank': [6.0,3.0,4.0,1.0,5.0,2.0],
        'nearest_distance_rank': [5.0,3.0,4.0,6.0,1.0,2.0],
        'transit_center_rank': [6.0,'NaN',5.0,'NaN',6.0,'NaN']
       }

rank_table = pd.DataFrame (data, 
        columns = ['availability_rank','SCR_rank','nearest_distance_rank','transit_center_rank'],
        index = ['Downtown West','Marcy Holmes','University','Cedar Riverside','North Loop','Prospect Park']
        )
rank_table

,availability_rank,SCR_rank,nearest_distance_rank,transit_center_rank
Downtown West,6.0,6.0,5.0,6
Marcy Holmes,5.0,3.0,3.0,NaN
University,4.0,4.0,4.0,5
Cedar Riverside,3.0,1.0,6.0,NaN
North Loop,2.0,5.0,1.0,6
Prospect Park,1.0,2.0,2.0,NaN


In [9]:
rank_table['average'] = rank_table.mean(numeric_only=True, axis=1)
rank_table

,availability_rank,SCR_rank,nearest_distance_rank,transit_center_rank,average
Downtown West,6.0,6.0,5.0,6,5.666667
Marcy Holmes,5.0,3.0,3.0,NaN,3.666667
University,4.0,4.0,4.0,5,4.000000
Cedar Riverside,3.0,1.0,6.0,NaN,3.333333
North Loop,2.0,5.0,1.0,6,2.666667
Prospect Park,1.0,2.0,2.0,NaN,1.666667


In [10]:
rank_table2 = rank_table.sort_values(by=['average'], ascending=False)
rank_table2

,availability_rank,SCR_rank,nearest_distance_rank,transit_center_rank,average
Downtown West,6.0,6.0,5.0,6,5.666667
University,4.0,4.0,4.0,5,4.000000
Marcy Holmes,5.0,3.0,3.0,NaN,3.666667
Cedar Riverside,3.0,1.0,6.0,NaN,3.333333
North Loop,2.0,5.0,1.0,6,2.666667
Prospect Park,1.0,2.0,2.0,NaN,1.666667


Based on the ranking table above the best neighborhood to place a scooter maintenance facility based upon our criteria is Downtown West. The next best neighborhood would be Univeristy, etc...